**Instituto de Informática - UFG**

Disciplina: Inteligência Computacional

Prof. Cedric Luiz de Carvalho


Grupo ( J )

---


Analista de dados:
* Arthur Cavalcante de Andrade
* William Teixeira Pires Junior

---

Engenheiro(a) do conhecimento:
* Arthur Cavalcante de Andrade
* William Teixeira Pires Junior

---

# Modelo

Para realizar a leitura da base de dados, atribuimos a variável `data`:

In [2]:
import pandas

# leitura do arquivo .csv
data = pandas.read_csv('PS_20174392719_1491204439457_log.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


A base de dados apresenta 11 colunas, que indicam:
* `step`: em horas, desde o inicio da observação, o momento em que a transação ocorre;
* `type`: tipo de transação;
* `amount`: valor envolvido na transação;
* `nameOrig`: nome da conta que realizou a transação;
* `oldbalanceOrg`: balanço da conta de origem da transação antes da operação ser realizada;
* `newbalanceOrig`: balanço da conta de origem da transação após a operação ser realizada;
* `nameDest`: nome da conta de destino da transação;
* `oldbalanceDest`: balanço da conta de destino da transação antes da operação ser realizada;
* `newbalanceDest`: balanço da conta de destino da transação após a operação ser realizada;
* `isFraud`: se a operação foi realizada por um agente fraudulento;
* `isFlaggedFraud`: se a operação movimenta um valor acima de 200 mil em uma única transação.

Assim como na base de dados, consideraremos **`fraude`** como sendo o comportamento de tomar a conta de um cliente e então transferir todo o dinheiro para uma outra conta que então realizará o saque.

Como não sabemos se a transação está sendo realizada pelo dono da conta ou pelo agente fraudulento, a árvore de decisão deverá identificar um comportamento fraudulento em cada transação de forma isolada, sem levar em conta movimentações prévias.

#### Mais descrições sobre o modelo são feitas nos outros notebooks.